In [1]:
# Importing our dependencies
import csv
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Reading our CSV file and putting it into a variable
tokyo = pd.read_csv("Resources/summary_listings.csv")
tokyo.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,197677,Rental unit in Sumida · ★4.78 · 1 bedroom · 2 ...,964081,Yoshimi & Marek,NaN,Sumida Ku,35.71707,139.82608,Entire home/apt,11000,3,173,2023-05-30,1.21,1,24,8,M130003350
1,776070,Home in Kita-ku · ★4.98 · 1 bedroom · 1 bed · ...,801494,Kei,NaN,Kita Ku,35.73844,139.76917,Private room,7208,3,243,2023-06-20,1.89,1,67,15,NaN
2,3427384,Rental unit in Edogawa · ★4.82 · 1 bedroom · 2...,13018876,Masakatsu,NaN,Edogawa Ku,35.68374,139.85971,Entire home/apt,7847,2,100,2023-05-22,0.93,2,231,19,Hotels and Inns Business Act | 東京都江戸川区保健所 | 18...
3,905944,Rental unit in Shibuya · ★4.76 · 2 bedrooms · ...,4847803,Best Stay In Tokyo!,NaN,Shibuya Ku,35.67878,139.67847,Entire home/apt,23066,3,186,2023-06-26,1.49,5,229,1,Hotels and Inns Business Act | 渋谷区保健所長 | 31渋健生...
4,3514008,Rental unit in Arakawa-ku · ★4.86 · 1 bedroom ...,17694529,Hisao,NaN,Arakawa Ku,35.72672,139.78201,Entire home/apt,2871,1,269,2023-06-08,2.59,9,11,29,Hotels and Inns Business Act | 東京都荒川区保健所 | 31荒...


In [3]:
# Deleting unwanted columns and making sure it stays in place
columns_to_delete = ['neighbourhood_group', 'last_review', 'calculated_host_listings_count', 
                     'number_of_reviews_ltm', 'license']
tokyo.drop(columns=columns_to_delete, inplace=True)

tokyo.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365
0,197677,Rental unit in Sumida · ★4.78 · 1 bedroom · 2 ...,964081,Yoshimi & Marek,Sumida Ku,35.71707,139.82608,Entire home/apt,11000,3,173,1.21,24
1,776070,Home in Kita-ku · ★4.98 · 1 bedroom · 1 bed · ...,801494,Kei,Kita Ku,35.73844,139.76917,Private room,7208,3,243,1.89,67
2,3427384,Rental unit in Edogawa · ★4.82 · 1 bedroom · 2...,13018876,Masakatsu,Edogawa Ku,35.68374,139.85971,Entire home/apt,7847,2,100,0.93,231
3,905944,Rental unit in Shibuya · ★4.76 · 2 bedrooms · ...,4847803,Best Stay In Tokyo!,Shibuya Ku,35.67878,139.67847,Entire home/apt,23066,3,186,1.49,229
4,3514008,Rental unit in Arakawa-ku · ★4.86 · 1 bedroom ...,17694529,Hisao,Arakawa Ku,35.72672,139.78201,Entire home/apt,2871,1,269,2.59,11


In [4]:
# Splitting 'name' column by '·'
tokyo[['name', 'ratings', 'bedrooms', 'beds', 'baths']] = tokyo['name'].str.split('·', expand=True)

#Removing the star from the ratings row
def remove_star(value):
    if isinstance(value, str):
        return value.replace('★', '').strip()
    return value
        
tokyo = tokyo.applymap(remove_star)
tokyo.head()

/var/folders/6z/lnz45jj5627_h_mynrn_15rr0000gn/T/ipykernel_21129/3849390388.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tokyo = tokyo.applymap(remove_star)


,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365,ratings,bedrooms,beds,baths
0,197677,Rental unit in Sumida,964081,Yoshimi & Marek,Sumida Ku,35.71707,139.82608,Entire home/apt,11000,3,173,1.21,24,4.78,1 bedroom,2 beds,1 bath
1,776070,Home in Kita-ku,801494,Kei,Kita Ku,35.73844,139.76917,Private room,7208,3,243,1.89,67,4.98,1 bedroom,1 bed,1 shared bath
2,3427384,Rental unit in Edogawa,13018876,Masakatsu,Edogawa Ku,35.68374,139.85971,Entire home/apt,7847,2,100,0.93,231,4.82,1 bedroom,2 beds,1.5 baths
3,905944,Rental unit in Shibuya,4847803,Best Stay In Tokyo!,Shibuya Ku,35.67878,139.67847,Entire home/apt,23066,3,186,1.49,229,4.76,2 bedrooms,4 beds,1 bath
4,3514008,Rental unit in Arakawa-ku,17694529,Hisao,Arakawa Ku,35.72672,139.78201,Entire home/apt,2871,1,269,2.59,11,4.86,1 bedroom,2 beds,1 bath


In [5]:
##Checking the object types
tokyo.dtypes

id                     int64
name                  object
host_id                int64
host_name             object
neighbourhood         object
latitude             float64
longitude            float64
room_type             object
price                  int64
minimum_nights         int64
number_of_reviews      int64
reviews_per_month    float64
availability_365       int64
ratings               object
bedrooms              object
beds                  object
baths                 object
dtype: object

In [6]:
#Taking out values od 0 or that are blank
tokyo.replace(0, pd.NA, inplace=True)
tokyo.dropna(axis=0, how='any', inplace=True)
tokyo.head(10)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365,ratings,bedrooms,beds,baths
0,197677,Rental unit in Sumida,964081,Yoshimi & Marek,Sumida Ku,35.717070,139.826080,Entire home/apt,11000,3,173,1.21,24,4.78,1 bedroom,2 beds,1 bath
1,776070,Home in Kita-ku,801494,Kei,Kita Ku,35.738440,139.769170,Private room,7208,3,243,1.89,67,4.98,1 bedroom,1 bed,1 shared bath
2,3427384,Rental unit in Edogawa,13018876,Masakatsu,Edogawa Ku,35.683740,139.859710,Entire home/apt,7847,2,100,0.93,231,4.82,1 bedroom,2 beds,1.5 baths
3,905944,Rental unit in Shibuya,4847803,Best Stay In Tokyo!,Shibuya Ku,35.678780,139.678470,Entire home/apt,23066,3,186,1.49,229,4.76,2 bedrooms,4 beds,1 bath
4,3514008,Rental unit in Arakawa-ku,17694529,Hisao,Arakawa Ku,35.726720,139.782010,Entire home/apt,2871,1,269,2.59,11,4.86,1 bedroom,2 beds,1 bath
5,1016831,Home in Setagaya,5596383,Wakana,Setagaya Ku,35.658000,139.671340,Private room,16000,1,244,1.96,101,4.94,1 bedroom,2 beds,1 shared bath
6,3563698,Rental unit in Machida,11583876,Ken,Machida Shi,35.589110,139.480230,Entire home/apt,4814,3,120,1.11,224,4.87,1 bedroom,2 beds,1 bath
8,3572139,Home in Toshima-ku,15514130,Tomo,Toshima Ku,35.727843,139.734084,Entire home/apt,31229,3,201,2.10,196,4.85,3 bedrooms,6 beds,1 bath
9,1249571,Home in Shibuya,6809418,Yume,Shibuya Ku,35.651110,139.721650,Entire home/apt,54782,7,116,0.97,189,4.96,1 bedroom,1 bed,1 bath
10,3623844,Rental unit in Edogawa,13018876,Masakatsu,Edogawa Ku,35.683580,139.859850,Entire home/apt,7538,2,77,0.77,210,4.86,1 bedroom,4 beds,1.5 baths


In [8]:
# Changing the types to float from object for plotting.
tokyo['ratings'] = tokyo['ratings'].str.extract('(\d+\.\d+|\d+)').astype(float)
tokyo['bedrooms'] = tokyo['bedrooms'].str.extract('(\d+)').astype(float)
tokyo['beds'] = tokyo['beds'].str.extract('(\d+)').astype(float)
tokyo['baths'] = tokyo['baths'].str.extract('(\d+)').astype(float)
tokyo.dtypes

id                     int64
name                  object
host_id                int64
host_name             object
neighbourhood         object
latitude             float64
longitude            float64
room_type             object
price                 object
minimum_nights         int64
number_of_reviews     object
reviews_per_month    float64
availability_365      object
ratings              float64
bedrooms             float64
beds                 float64
baths                float64
dtype: object

In [9]:
# Reorganizing columns into order of identification and location, host information, property information, booking and review information, and rating and accomodation details
new_column_order = [
    'id', 'name', 'neighbourhood', 'latitude', 'longitude',
    'host_id', 'host_name', 'room_type', 'price',
    'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'availability_365',
    'ratings', 'bedrooms', 'beds', 'baths'
]

tokyo_df = tokyo[new_column_order]
tokyo_df.head()

,id,name,neighbourhood,latitude,longitude,host_id,host_name,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365,ratings,bedrooms,beds,baths
0,197677,Rental unit in Sumida,Sumida Ku,35.71707,139.82608,964081,Yoshimi & Marek,Entire home/apt,11000,3,173,1.21,24,4.78,1.0,2.0,1.0
1,776070,Home in Kita-ku,Kita Ku,35.73844,139.76917,801494,Kei,Private room,7208,3,243,1.89,67,4.98,1.0,1.0,1.0
2,3427384,Rental unit in Edogawa,Edogawa Ku,35.68374,139.85971,13018876,Masakatsu,Entire home/apt,7847,2,100,0.93,231,4.82,1.0,2.0,1.0
3,905944,Rental unit in Shibuya,Shibuya Ku,35.67878,139.67847,4847803,Best Stay In Tokyo!,Entire home/apt,23066,3,186,1.49,229,4.76,2.0,4.0,1.0
4,3514008,Rental unit in Arakawa-ku,Arakawa Ku,35.72672,139.78201,17694529,Hisao,Entire home/apt,2871,1,269,2.59,11,4.86,1.0,2.0,1.0


In [10]:
tokyo_df_organized = pd.DataFrame(tokyo_df)
csv_path = 'Resources/updated_summarylist.csv'
tokyo_df_organized.to_csv(csv_path, index = False)

In [12]:
# Create sqlite file
path = 'Resources/tokyoAirbnb.sqlite'
conn = sqlite3.connect(path)

# Write the DataFrame to the database
tokyo_df_organized.to_sql(name='tokyoAirbnb',
                          con=conn,
                          if_exists='replace',
                          index=False,
                          dtype={'id':'INTEGER PRIMARY KEY',
                                'name':'STRING',
                                'neighbourhood':'TEXT',
                                'latitude':'FLOAT',
                                'longitude':'FLOAT',
                                'host_id': 'INTEGER',
                                'host_name':'TEXT',
                                'room_type':'TEXT',
                                'price':'INTEGER',
                                'minimum_nights':'INTEGER',
                                'number_of_reviews':'INTEGER',
                                'reviews_per_month':'FLOAT',
                                'availability_365':'INTEGER',
                                'ratings':'FLOAT',
                                'bedrooms':'FLOAT',
                                'beds':'FLOAT',
                                'baths':'FLOAT'})

# Close the connection
conn.close()